In [109]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
import regex
import itertools
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [110]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [111]:
url = "https://www.ifebp.org/education/virtual-conferences/Pages/default.aspx"

In [112]:
driver.get(url)

In [113]:
try:
    driver.maximize_window()
except:
    pass

In [114]:
html = driver.find_element_by_xpath('//*[@id="WebPartctl00_ctl44_g_fc835a5e_1714_4b14_8ddb_8bbb82570e72"]').get_attribute('outerHTML')

In [115]:
soup = bs(html,'lxml')

In [116]:
all_texts = soup.find_all('div',{'class':re.compile('epContainer.+')})

In [117]:
len(all_texts)

4

In [144]:
ifebp = []
for k in all_texts:
    date__ = k.find('span',{'class':'epDates'}).text#.find('span').text
    tit = k.find('div',{'class':'epInfo'}).find('a').text
    link=k.find('a')['href']
    event_w = k.find('div',{'class':'epButtons'}).find('a')['href']
    driver.get(link)
    soupn = bs(driver.page_source,'lxml')

    event_info=' '.join(soupn.find('div',{'id':'DeltaPlaceHolderMain'}).find('div',{'class':'intro-wrap'}).find('div',{'class':'ms-rtestate-field'}).text.split())
    if event_info=='':
        event_info=' '.join(soupn.find('div',{'id':'DeltaPlaceHolderMain'}).find('div',{'class':'intro-wrap'}).find('div',{'class':re.compile('ms-webpart.+')}).text.split())

    #test
    sesame=soupn.find('div',{'id':'DeltaPlaceHolderMain'}).findChild()['class']

    if 'layout-wrap' in sesame:
        title=' '.join(soupn.find('div',{'id':'DeltaPlaceHolderMain'}).find('div',{'class':'title-date-loc'}).h1.text.split())
        datiloc=' '.join(soupn.find('div',{'id':'progDateLoc'}).get_text(separator=']').split())
    else:
        title=' '.join(soupn.find('div',{'id':'DeltaPlaceHolderMain'}).find('div',{'class':'header_info'}).h1.text.split())
        datiloc=' '.join(soupn.find('div',{'id':'progDateLoc'}).get_text(separator=']').split())

    dat,loc=datiloc[:-1].split(']')
    dat=dat.replace(',','').replace('–','-')
    for f in ['Monday','Tuesday','Wednesday','Thursday','Fiday','Saturday','Sunday']:
        if f in dat:
            dat=dat.replace(f,'')#.strip()
    mad=dat
    try:
        if '-' in mad:
            st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

        elif 'and'in mad:
            st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

        elif '&' in mad:
            st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

        else:
            st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        #print(date__,st,en)
        ####################
        if any(c.isalpha() for c in st)==False:
        #      print('leg')
            pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
            sapa=pa.group()
            start=st+' '+sapa
            end=en
        #                        02        April      2022
        elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
         #   print('awujale')
            start=st
            end=en

        #*                        April       02      2022
        elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
        #       print('ala')
            pa=re.search('(\d{4})',st).group()
            sa=re.search('([A-Sa-z]{3,9})',st).group()
            ta=re.search('(\d{1,2})',st).group()
            start=ta+' '+sa+' '+pa
            #
            ba=re.search('(\d{4})',en).group()
            ca=re.search('([A-Sa-z]{3,9})',en).group()
            da=re.search('(\d{1,2})',en).group()
            end=da+' '+ca+' '+ba     

        #                      02     2022
        elif re.search('\d{1,2}\s+\d{4}',en):
        #      print('kum')
            pa=re.search('(\d{4})',en).group()#, maxsplit=0)
            sa=re.search('([A-Sa-z]{3,9})',st).group()
            ta=re.search('(\d{1,2})',st).group()

            start=ta+' '+sa+' '+pa
            #
            ba=re.search('(\d{4})',en).group()
            ca=re.search('([A-Sa-z]{3,9})',st).group()
            da=re.search('(\d{1,2})',en).group()
            end=da+' '+ca+' '+ba

        #                      02         April
        elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
        #        print('is')
            pa=re.search('(\d{4})',en)#, maxsplit=0)
            sapa=pa.group()
            start=st+' '+sapa
            end=en

        #*                      April           02
        elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
        #      print('bad')
            pa=re.search('(\d{4})',en).group()#, maxsplit=0)
            sa=re.search('([A-Sa-z]{3,9})',st).group()
            ta=re.search('(\d{1,2})',st).group()

            start=ta+' '+sa+' '+pa
            #
            ba=re.search('(\d{4})',en).group()
            ca=re.search('([A-Sa-z]{3,9})',en).group()
            da=re.search('(\d{1,2})',en).group()
            end=da+' '+ca+' '+ba

        else:
        #     print('shik')
            start=end=''
        if start=='':
            start_date=end_date=''
        else:
            pick=[start, end]
            spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
            date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
            start_date=date_[0]
            end_date=date_[1]
    except:
        start_date=end_date=''

    opor=' '.join(soupn.find('div',{'class':'registration-info-wrap'}).find('table').tbody.find_all('tr')[2].text.split())#.split()#.get_text(separator='').split())
    opot=opor.replace(',','')
    prices=regex.findall('\p{Sc}\d+\W?\d+',opot)#.group().replace(',','')

    if len(prices)==1:
        po=prices[0]
    else:
        po=prices[1]
    cu=po[:1]
    if float(po[1:])>0:
        ty='paid'
        cu=cu
        pr=po[1:]
    else:
        ty='Free'
        cu=''
        pr=''
    if ty=='':
        ticket_list=''
    else:
        ticketlist={'type': ty,
                     'price':pr,
                     'currency': cu}
        ticket_list=json.dumps([ticketlist])
        
    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('ZMeLFc').click()
                except:
                    
#                     try:
#                         google_map_url=driver.find_element_by_class_name('o8ebK').click()
#                     except:
                    google_map_url=driver.find_element_by_class_name('o8ebK').click()
            time.sleep(1)
            try:
                driver.find_element_by_class_name('k7jAl miFGmb')
                try:
                    country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
                except:
                    try:
                        country=driver.find_element_by_class_name('bwoZTb').text
                    except:
                        country=driver.find_element_by_class_name('rogA2c').text.split(',')[-1].strip()
            except:
                try:
                    driver.find_element_by_class_name('cXedhc').click()
                    time.sleep(3)
                    country=' '.join(driver.find_element_by_class_name('Z1hOCe').text.split()).split(',')[-1].strip()
                except:
                    try:
                        country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
                    except:
                        try:
                            country=driver.find_element_by_class_name('bwoZTb').text
                        except:
                            country=driver.find_element_by_class_name('rogA2c').text.split(',')[-1].strip()
    #                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")

    ####################################
    def get_google_map_url(location):
        try:
            google_url_for_location="https://www.google.com/search?q="+location+"&oq="+location+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('ZMeLFc').click()
                except:
                    google_map_url=driver.find_element_by_class_name('o8ebK').click()
            time.sleep(1)
            try:
                driver.find_element_by_class_name('k7jAl miFGmb')
                google_map_url=driver.current_url
            except:
                try:
                    driver.find_element_by_class_name('cXedhc').click()
                    time.sleep(3)
                    google_map_url=driver.current_url
                except:
                    google_map_url=driver.current_url
#                 print(google_map_url)
            return(google_map_url)
        except Exception as e:
            print(location, "; url didn't go through")
            print(e)
            return("")
    ##########################################
    if loc=='':
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        on_off='0'
        venue=loc
        city=loc
        country=countr(city)
        googlePlaceUrl=get_google_map_url(venue)
        
    org_pro=' '.join("""The International Foundation of Employee Benefit Plans is a nonprofit organization, dedicated 
                        to providing the diverse employee benefits community with objective, solution-oriented education,
                        research and information.""".split())
    org_web='https://www.ifebp.org/'
    org_name='International Foundation of Employee Benefit Plans'
    
    mail_=''
    logo=''
    sponsors=''
    agendalist=''
    type_=''
    category=''
    Speakerlist=''
    event_web=link
    time_=''
    ticket_list=''
    event_web=link
    ifebp.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsors,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])
    
print('Done')

Done


In [145]:
ifebp_dict=ifebp


ifebp_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=ifebp_dict)
ifebp_df.to_csv("ifebp.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [ ]:
# df = df.to_json(orient='split')

In [ ]:
# ash="""    ALABAMA	AL
#     ALASKA	AK
#     AMERICAN SAMOA	AS
#     ARIZONA	AZ
#     ARKANSAS	AR
#     CALIFORNIA	CA
#     COLORADO	CO
#     CONNECTICUT	CT
#     DELAWARE	DE
#     DISTRICT OF COLUMBIA	DC
#     FLORIDA	FL
#     GEORGIA	GA
#     GUAM	GU
#     HAWAII	HI
#     IDAHO	ID
#     ILLINOIS	IL
#     INDIANA	IN
#     IOWA	IA
#     KANSAS	KS
#     KENTUCKY	KY
#     LOUISIANA	LA
#     MAINE	ME
#     MARYLAND	MD
#     MASSACHUSETTS	MA
#     MICHIGAN	MI
#     MINNESOTA	MN
#     MISSISSIPPI	MS
#     MISSOURI	MO
#     MONTANA	MT
#     NEBRASKA	NE
#     NEVADA	NV
#     NEW HAMPSHIRE	NH
#     NEW JERSEY	NJ
#     NEW MEXICO	NM
#     NEW YORK	NY
#     NORTH CAROLINA	NC
#     NORTH DAKOTA	ND
#     NORTHERN MARIANA IS	MP
#     OHIO	OH
#     OKLAHOMA	OK
#     OREGON	OR
#     PENNSYLVANIA	PA
#     PUERTO RICO	PR
#     RHODE ISLAND	RI
#     SOUTH CAROLINA	SC
#     SOUTH DAKOTA	SD
#     TENNESSEE	TN
#     TEXAS	TX
#     UTAH	UT
#     VERMONT	VT
#     VIRGINIA	VA
#     VIRGIN ISLANDS	VI
#     WASHINGTON	WA
#     WEST VIRGINIA	WV
#     WISCONSIN	WI
#     WYOMING	WY  """

In [ ]:
# shigan=ash.replace('    ', '').split('\n')
# shi={q.split('\t')[1].strip() : q.split('\t')[0].strip() for q in shigan}

In [ ]:
# shi.get('WY')

In [ ]:
#driver.switch_to.window(driver.window_handles[4])